## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 1: Quick Start

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import os
import asyncio
import logging
from typing import Annotated

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

In [2]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

### 🪜 Step 2: Add AI services

In [3]:
# Initialise kernel
kernel = Kernel()

In [4]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
)

kernel.add_service(chat_completion)

### 🪜 Step 3: Enable logging

In [5]:
# Set logging level for SK to DEBUG.
logging.basicConfig(
    format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.getLogger("kernel").setLevel(logging.DEBUG)

### 🪜 Step 4: Add plugin

In [6]:
# Define lights plugin
class LightsPlugin:
    lights = [
        {"id": 1, "name": "Table Lamp", "is_on": False},
        {"id": 2, "name": "Porch light", "is_on": False},
        {"id": 3, "name": "Chandelier", "is_on": True},
    ]

    @kernel_function(
        name="get_lights",
        description="Gets a list of lights and their current state",
    )
    def get_state(
        self,
    ) -> str:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function(
        name="change_state",
        description="Changes the state of the light",
    )
    def change_state(
        self,
        id: int,
        is_on: bool,
    ) -> str:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = is_on
                return light
        return None

In [7]:
# Add plugin to the kernel
kernel.add_plugin(
    LightsPlugin(),
    plugin_name="Lights",
)

KernelPlugin(name='Lights', description=None, functions={'change_state': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='change_state', plugin_name='Lights', description='Changes the state of the light', parameters=[KernelParameterMetadata(name='id', description=None, default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer'}, include_in_function_choices=True), KernelParameterMetadata(name='is_on', description=None, default_value=None, type_='bool', is_required=True, type_object=<class 'bool'>, schema_data={'type': 'boolean'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyH

### 🪜 Step 5: Define behaviour

In [8]:
# Auto-invoke function calling
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

### 🪜 Step 6: Run the process

In [9]:
# Create conversation history
history = ChatHistory()

In [10]:
# Helper function to get AI response
async def get_response(prompt):
    # Add the prompt to the history
    history.add_user_message(prompt)

    # Get the response from the AI
    result = (await chat_completion.get_chat_message_contents(
        chat_history = history,
        settings = execution_settings,
        kernel = kernel,
        arguments = KernelArguments(),
    ))[0]

    # Add the response to the history
    history.add_message(result)

    return result

In [11]:
# Test AI function calling
queries = [
    "Get the lights state",
    "Change the state of the table lamp to on",
    "Get the lights state",
    "Change the state of the porch light to on",
    "Get the lights state",
    "Change the state of the chandelier to off",
    "Get the lights state",
]

for query in queries:
    print(f"User: {query}")
    response = await get_response(query)
    print(f"AI: {response}\n")

User: Get the lights state
AI: There are three lights with the following states:
- Table Lamp: Off
- Porch light: Off
- Chandelier: On

Would you like to change the state of any of these lights?

User: Change the state of the table lamp to on
AI: The Table Lamp has been turned on. Is there anything else you'd like to do?

User: Get the lights state
AI: The current state of the lights is:
- Table Lamp: On
- Porch light: Off
- Chandelier: On

Let me know if you want to change anything or need further assistance.

User: Change the state of the porch light to on
AI: The Porch light has been turned on. If you need any more changes or information, feel free to ask!

User: Get the lights state
AI: All the lights are currently on:
- Table Lamp: On
- Porch light: On
- Chandelier: On

Is there anything else you would like to do?

User: Change the state of the chandelier to off
AI: The Chandelier has been turned off. If you need any further assistance, please let me know!

User: Get the lights st